In [1]:
import tensorflow as tf

2024-10-20 08:45:08.768378: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-20 08:45:08.781555: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-20 08:45:08.797738: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-20 08:45:08.802613: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-20 08:45:08.814182: I tensorflow/core/platform/cpu_feature_guar

In [2]:
tf.__version__

'2.17.0'

In [3]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]

In [4]:
# install gdown
# !pip install gdown

In [5]:
# download dataset file with gdown
import gdown
url = "https://drive.google.com/uc?id=1p8Flgeg-pd1pNdc4-KMO3vKlT8XLc1wt"
output = "dataset.zip"
gdown.download(url=url, output=output)

Downloading...
From (original): https://drive.google.com/uc?id=1p8Flgeg-pd1pNdc4-KMO3vKlT8XLc1wt
From (redirected): https://drive.google.com/uc?id=1p8Flgeg-pd1pNdc4-KMO3vKlT8XLc1wt&confirm=t&uuid=9d5504b1-79e1-4505-afad-28b581226124
To: /home/ibnuk/algoritma-deep-learning-gundar/dataset.zip
100%|██████████| 455M/455M [00:02<00:00, 198MB/s] 


'dataset.zip'

In [7]:
# unzip dataset file
import zipfile
with zipfile.ZipFile(output, 'r') as zip_ref:
    zip_ref.extractall("dataset")